<a href="https://colab.research.google.com/github/jhmuller/nextWord/blob/main/nextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import requests
import re
import string
import numpy as np
import sklearn

https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf

In [97]:
URL = 'https://www.gutenberg.org/files/67138/67138-0.txt'
response = requests.get(URL)
orig_text = response.content

In [123]:
M = 2535
temp_text = str(orig_text[M:])
clean_text = re.sub("\\\\r|\\\\n|\\\\xbb|\\\\xef|\\\\xbf|x80|x9|xe|\d.", " ", temp_text)
cleaner_text = clean_text.translate(str.maketrans('', '', string.punctuation))

In [124]:
len(cleaner_text)
cleaner_text[:200]

'b    Robert Cohn was once middleweight boxing champion of Princeton Do not  think that I am very much impressed by that as a boxing title but it  meant a lot to Cohn He cared nothing for boxing in fac'

In [127]:
 
N = len(cleaner_text)
cutoff = int(np.floor(0.4*N))
while cleaner_text[cutoff] != ' ':
  cutoff += 1
train = cleaner_text[:cutoff]
test = cleaner_text[cutoff:]
test[:100]

' from them The green plain stretched  off It was cut by fences and the white of the road showed thro'

In [128]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts([train])

def make_Xy(data, tokenizer, ngram_len=1):

  sequence_data = tokenizer.texts_to_sequences([data])[0]
  print(len(sequence_data))
  sequences = []
  X = []
  y = []
  for i in range(1, len(sequence_data)-ngram_len-1):
      words = sequence_data[i-1:i+ngram_len]
      X.append(words[:-1])
      y.append(words[-1])

  X = np.array(X)
  y = np.array(y)

  from  tensorflow.keras.utils import to_categorical
  vocab_size = len(tokenizer.word_index) + 1

  ycat = to_categorical(y, num_classes=vocab_size)
  return X, ycat, y

Xtrain, ytrain, yindstrain = make_Xy(train, tokenizer, ngram_len=3)
Xtest, ytest, yindstest = make_Xy(test, tokenizer, ngram_len=3)



29962
39497


In [129]:
print(tokenizer.sequences_to_texts([yindstest[:5]]))
tokenizer.sequences_to_texts(Xtest[:5])

['green plain stretched off it']


['from them the',
 'them the green',
 'the green plain',
 'green plain stretched',
 'plain stretched off']

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
def get_vocab_size(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([data])
  vocab_size = len(tokenizer.word_index) + 1
  return vocab_size

train_vocab_size = get_vocab_size(train)
model = Sequential()
model.add(Embedding(train_vocab_size, 50, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(train_vocab_size, activation="softmax"))

In [131]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer=Adam(lr=0.001))
model.fit(Xtrain, ytrain, epochs=10, batch_size=64, callbacks=[reduce_lr, tensorboard_Visualization])

Epoch 1/10
469/469 [==============================] - 9s 14ms/step - loss: 6.2562 - accuracy: 0.0509 - lr: 0.0010
Epoch 2/10
469/469 [==============================] - 6s 13ms/step - loss: 5.8083 - accuracy: 0.0709 - lr: 0.0010
Epoch 3/10
469/469 [==============================] - 6s 14ms/step - loss: 5.4204 - accuracy: 0.1050 - lr: 0.0010
Epoch 4/10
469/469 [==============================] - 6s 14ms/step - loss: 5.0590 - accuracy: 0.1312 - lr: 0.0010
Epoch 5/10
469/469 [==============================] - 6s 13ms/step - loss: 4.7680 - accuracy: 0.1494 - lr: 0.0010
Epoch 6/10
436/469 [==========================>...] - ETA: 0s - loss: 4.5271 - accuracy: 0.1670

In [93]:
vpreds = model.predict(Xtest[:10])
print(len(vpreds))
pred_inds = [np.argmax(x) for x in vpreds]
pred_words = tokenizer.sequences_to_texts([pred_inds])
pred_words

1/1 [==============================] - 1s 619ms/step
10


['mountains glass the big it on saw to any crowd']

In [87]:
true_inds = [np.argmax(x) for x in ytest[:10]]
true_words = tokenizer.sequences_to_texts([true_inds])
true_words

['one at a time and they have in the to']

In [89]:
true_inds

[37, 15, 4, 101, 2, 50, 33, 9, 1, 5]

In [94]:
print(vpreds[1][15], max(vpreds[1]))

0.002098431 0.40241668


In [95]:
pred_inds

[390, 247, 1, 122, 11, 14, 151, 5, 77, 376]